<a href="https://colab.research.google.com/github/liuyao12/imagenette_experiments/blob/master/ResNet_separable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet with a Twist, with depthwise separable convolution

> depthwise (x4) + Ranger + Mish + SA + MaxBlurPool + ResTrick

See blog https://liuyao12.github.io/blog/research/2020/03/07/Conv-Twist.html

See summary at https://forums.fast.ai/t/imagenette-imagewoof-leaderboards/45822/47?u=liuyao 

## Imagewoof Leaderboard

Imagewoof2, with a 70/30 train/test ratio.

| Size (px) | Epochs | x2 | x4 | x6 | x4 double | runs |
|--|--|--| --|--| --|--|
|128|5|75.19|76.27|76.61| **82.12**|5, mean
|128|20|85.18|86.22|86.27| **88.93**|5, mean
|128|80|87.70|87.83|87.65| **90.15**|1
|128|200|
|192|5|79.86|81.15|| **82.69**|5, mean
|192|20|88.12|88.37|
|192|80|90.30|89.89|| **92.08** |
|192|200|
|256|5|
|256|20|
|256|80|
|256|200|


# setup and imports

In [ ]:
# pip install kornia

In [ ]:
pip install git+https://github.com/ayasyrev/model_constructor

In [ ]:
pip install git+https://github.com/ayasyrev/imagenette_experiments

In [4]:
from fastai.basic_train import *
from fastai.vision import *
# from fastai.script import *

In [5]:
from kornia.contrib import MaxBlurPool2d

In [6]:
from imagenette_experiments.train_utils import *
from model_constructor.net import Net, act_fn
from model_constructor.layers import SimpleSelfAttention, ConvLayer

# ResBlock

In [17]:
class MnM(nn.Module): # Mix and Multiply
    def __init__(self, channels, group_size):
        super().__init__()
        self.channels = channels
        self.gs = group_size
        self.conv = nn.Conv2d(channels, channels, 1, groups=1, bias=False)
        self.mask = torch.zeros((channels,channels,1,1))
        for i in range(channels//group_size):
            self.mask[i*dm:(i+1)*dm,i*dm:(i+1)*dm] = 1
        self.XY = None

    def forward(self, x): 
        if self.XY is None:
            N,C,H,W = x.size()
            XX = torch.from_numpy(np.indices((1,H,W))[2]*2/W-1)
            YY = torch.from_numpy(np.indices((1,H,W))[1]*2/H-1)
            ones = torch.ones((self.gs-2,H,W))
            XY = torch.cat([ones, XX, YY], dim=0).to(x.device).type(x.dtype)
            self.XY = XY.unsqueeze(0).expand(self.channels//self.gs,self.gs,H,W).reshape(self.channels,H,W)
            self.mask = self.mask.to(x.device).type(x.dtype)
        self.conv.weight.data = self.mask * self.conv.weight
        return self.conv(x) * self.XY

In [15]:
class NewLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1,
            act=True,  act_fn=nn.ReLU(inplace=True),
            bn_layer=True, bn_1st=True, zero_bn=False,
            padding=None, bias=False, groups=1, **kwargs):

        if padding==None: padding = ks//2
        if ks==3 and groups==1:  # to be used for the "stem" of ResNet
          # if ni==3: stride = 2
          layers = [('Conv3x3', nn.Conv2d(ni, ni*dm, 3, stride=stride, padding=1, bias=bias, groups=ni)),
                    ('Conv1x1', nn.Conv2d(ni*dm, nf, 1, bias=bias, groups=1))]
        else:
          layers = [('Conv{}x{}'.format(ks,ks), 
                      nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]
          if ks==3:
            layers += [('M_op', MnM(nf, dm))]

        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [9]:
class NewResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1,
                 conv_layer=ConvLayer, act_fn=act_fn, zero_bn=True, bn_1st=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False, sym=False, groups=1):
        nf,ni = nh*expansion,ni*expansion
        conv_layer = NewLayer
        self.reduce = noop if stride==1 else pool
        layers  = [# (f"conv_0", conv_layer(ni, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(ni, nf, 3, zero_bn=zero_bn, act=False, bn_layer=True))
        ] if expansion == 1 else [
                   (f"conv_0", conv_layer(ni, nh, 1, act_fn=act_fn, bn_1st=bn_1st)),
                   # (f"conv_1", conv_layer(nh, nh, 3, act_fn=act_fn, bn_1st=bn_1st)),
                   # (f"conv_1", conv_layer(nh, nh*dm, 3, groups=nh, act_fn=act_fn, bn_1st=bn_1st)),
                   (f"conv_1", conv_layer(nh, nh*dm, 3, groups=nh, act=False, bn_layer=False)),
                   (f"conv_2", conv_layer(nh*dm, nf, 1, zero_bn=zero_bn, act=False, bn_1st=bn_1st))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False, bn_1st=bn_1st)
        self.merge = act_fn

    def forward(self, x):
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

# Model Constructor

In [13]:
model = Net(c_out=10, layers=[3,6,8,3], expansion=4)
model.block = NewResBlock
model.conv_layer = NewLayer # for the stem
pool = MaxBlurPool2d(3, True)
model.pool = pool
model.stem_sizes = [3,32,64,64]
model.act_fn = Mish()
model.sa = True

## Experiment

In [ ]:
dm = 4
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.938423,1.738842,0.444388,0.895902,02:58
1,1.614446,1.593913,0.535760,0.921354,02:58
2,1.406542,1.312408,0.671163,0.955205,02:58
3,1.285317,1.198211,0.723848,0.967422,02:59
4,1.090849,1.036145,0.798422,0.980657,02:59


/usr/local/lib/python3.6/dist-packages/imagenette_experiments/train_utils.py:150: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


5 epochs: 0.7984219789505005 (1 runs)
data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.884023,1.835541,0.425045,0.888521,02:59
1,1.651628,1.466147,0.588954,0.939679,02:59


Start with a model with `*2` in all the depthwise convolutions.

In [ ]:
epochs = 60
mixup = 0 if epochs<=20 else 0.2
learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
print(learn.recorder.metrics[-1][0].item())

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.052636,1.938801,0.348944,0.834563,02:18
1,1.828844,1.628165,0.515398,0.912700,02:18
2,1.676960,1.429941,0.609061,0.935607,02:17
3,1.578073,1.276244,0.681344,0.958005,02:17
4,1.481553,1.214398,0.704250,0.957496,02:18
5,1.416507,1.187612,0.723339,0.965640,02:18
6,1.383856,1.080393,0.768389,0.967931,02:17
7,1.324159,1.050738,0.783405,0.973276,02:17
8,1.302396,1.006987,0.802749,0.975821,02:17
9,1.252993,0.984219,0.808603,0.976839,02:17


0.8979384303092957


In [ ]:
learn.validate()

[0.77321756, tensor(0.8979), tensor(0.9855)]

In [ ]:
state = learn.model.state_dict()
channels = 0
for name in state:
  if 'Conv3x3' in name and channels==0:
    a,b,c,d = state[name].size()
    double = state[name].unsqueeze(1).expand(a,2,b,c,d).reshape(a*2,b,c,d)
    state[name] = double / 1.414
    channels = a
  if 'Conv1x1' in name and state[name].size()[1]==channels:
    a,b,c,d = state[name].size()
    double = state[name].unsqueeze(2).expand(a,b,2,c,d).reshape(a,b*2,c,d)
    state[name] = double / 1.414
    channels = 0
print('done doubling')

done doubling


Here, manually change the `*2` to `*4` (in four places), rerun `model`, and proceed below:

In [ ]:
epochs = 10
mixup = 0 if epochs<=20 else 0.2
learn2 = get_learn(model=model, size=192, bs=16, mixup=mixup)
learn2.model.load_state_dict(state)
print(learn2.validate())

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


[0.7732239, tensor(0.8979), tensor(0.9855)]


In [ ]:
learn2.fit_fc(10, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
print(learn2.recorder.metrics[-1][0].item())

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,0.606666,0.875958,0.854416,0.978621,02:36
1,0.637473,0.875326,0.849835,0.977093,02:36
2,0.619375,0.890647,0.856707,0.975057,02:36
3,0.620187,0.899035,0.851107,0.979639,02:36
4,0.611394,0.852904,0.869941,0.979639,02:36
5,0.639448,0.892186,0.853907,0.979130,02:36
6,0.632713,0.866875,0.863324,0.975057,02:36
7,0.603388,0.837288,0.869687,0.984220,02:35
8,0.566116,0.818019,0.878595,0.981166,02:36
9,0.539309,0.791606,0.888012,0.982693,02:35


0.8880122303962708


In [ ]:
learn.validate()

NameError: ignored

In [ ]:
learn2.fit_fc(40, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,0.665684,0.900977,0.847289,0.977348,02:38
1,0.659895,0.883306,0.859506,0.976330,02:37
2,0.644420,0.890825,0.850344,0.980911,02:37
3,0.639653,0.889162,0.851871,0.976839,02:37
4,0.636691,0.873973,0.859506,0.978112,02:37
5,0.646427,0.874967,0.860270,0.975057,02:37
6,0.623019,0.871699,0.855943,0.977602,02:37
7,0.618884,0.850166,0.864851,0.980911,02:38
8,0.618632,0.862228,0.869432,0.978112,02:38
9,0.617541,0.883283,0.856452,0.977602,02:37


Buffered data was truncated after reaching the output size limit.

In [ ]:
for i in range(40):
  print(i, learn2.recorder.val_losses[i].item(), learn2.recorder.metrics[i][0].item())

0 0.9009769558906555 0.8472893834114075
1 0.8833058476448059 0.8595062494277954
2 0.8908252120018005 0.8503435850143433
3 0.8891615271568298 0.8518707156181335
4 0.8739725947380066 0.8595062494277954
5 0.8749666810035706 0.8602697849273682
6 0.8716986179351807 0.855942964553833
7 0.8501657247543335 0.8648511171340942
8 0.8622281551361084 0.8694324493408203
9 0.8832833766937256 0.8564520478248596
10 0.8733472228050232 0.8645966053009033
11 0.8668575286865234 0.8574700951576233
12 0.8748914003372192 0.8617969155311584
13 0.8977355360984802 0.8556884527206421
14 0.8786979913711548 0.8605242967605591
15 0.877534806728363 0.8567065596580505
16 0.8835002183914185 0.8589972257614136
17 0.88120436668396 0.8623059391975403
18 0.8919264078140259 0.8546704053878784
19 0.8665375113487244 0.8597607612609863
20 0.8747259378433228 0.8567065596580505
21 0.8497205376625061 0.8610333204269409
22 0.8672081828117371 0.8635784983634949
23 0.8595353364944458 0.868159830570221
24 0.8896135091781616 0.8495800

# results

## size=128


In [ ]:
dm = 4
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print({e: sum(res[e])/len(res[e]) for e in res})

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.961671,1.833559,0.419700,0.879104,01:43
1,1.671203,1.519507,0.545940,0.930008,01:42
2,1.506109,1.366296,0.635022,0.940443,01:43
3,1.357213,1.285933,0.673963,0.952151,01:43
4,1.169711,1.106144,0.759226,0.971749,01:42


{5: 0.7592262625694275}
data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.951077,1.792640,0.418682,0.872232,01:38
1,1.657203,1.508731,0.568083,0.932553,01:38
2,1.496707,1.367347,0.630949,0.943497,01:37
3,1.343047,1.288623,0.674472,0.951133,01:38
4,1.219010,1.204292,0.723085,0.963349,01:38
5,1.200550,1.127590,0.745228,0.972767,01:38
6,1.136331,1.137869,0.750318,0.965386,01:38
7,1.114759,1.074422,0.768134,0.972258,01:38
8,1.065833,1.027165,0.790277,0.973530,01:38
9,1.009803,0.997346,0.799949,0.977348,01:38


{5: 0.7622295618057251, 20: 0.8622550368309021}
data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.073898,1.859646,0.375159,0.856961,01:38
1,1.836540,1.617754,0.499618,0.918045,01:38
2,1.767781,1.572328,0.526597,0.915500,01:38
3,1.604483,1.273613,0.677781,0.952660,01:38
4,1.524711,1.220221,0.709341,0.957241,01:38
5,1.479169,1.163010,0.730975,0.965386,01:38
6,1.407726,1.153900,0.737592,0.964877,01:38
7,1.367976,1.072799,0.762026,0.973530,01:38
8,1.324972,1.059646,0.769152,0.974803,01:37
9,1.295897,1.040017,0.778061,0.975057,01:37


{5: 0.7622295618057251, 20: 0.8622550368309021, 80: 0.8783405423164368}


In [ ]:
print('depth multiplier =',dm)
print({e: sum(res[e])/len(res[e]) for e in res})

depth multiplier = 4
{5: 0.7622295618057251, 20: 0.8622550368309021, 80: 0.8783405423164368}


In [ ]:
dm = 6
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier='.format(dm), {e: sum(res[e])/len(res[e])})

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.924982,1.804799,0.412828,0.878595,01:47
1,1.682397,1.570771,0.531942,0.916773,01:47
2,1.513665,1.388963,0.625859,0.948842,01:46
3,1.349511,1.233298,0.690761,0.959786,01:46
4,1.168263,1.095365,0.760244,0.970985,01:47


5 epochs: 0.7602443099021912 (1 runs)
data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.907900,1.816525,0.401120,0.877577,01:46
1,1.671003,1.492498,0.557394,0.934843,01:46
2,1.491276,1.427671,0.603461,0.941461,01:46
3,1.374749,1.277938,0.679562,0.955968,01:46
4,1.286557,1.179975,0.730211,0.963604,01:46
5,1.238837,1.169291,0.727921,0.968694,01:47
6,1.135119,1.091888,0.769152,0.967931,01:46
7,1.139735,1.030810,0.784933,0.974294,01:46
8,1.057286,1.020697,0.797149,0.972512,01:46
9,1.009045,1.026826,0.797658,0.974803,01:47


20 epochs: 0.8627640604972839 (5 runs)
data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.104026,1.864537,0.367269,0.863833,01:46
1,1.856348,1.573903,0.525325,0.926444,01:47
2,1.680949,1.461086,0.586663,0.930517,01:47
3,1.594397,1.396894,0.609315,0.950115,01:47
4,1.553433,1.244519,0.683125,0.965386,01:47
5,1.474216,1.163294,0.733774,0.964622,01:47
6,1.433241,1.106275,0.758717,0.967167,01:47
7,1.356523,1.058523,0.779588,0.969203,01:47
8,1.332355,1.095734,0.750827,0.969203,01:47
9,1.298666,1.039442,0.773479,0.975312,01:47


80 epochs: 0.8765588998794556 (1 runs)
depth multiplier= {80: 0.8765588998794556}


In [ ]:
res[-5:], sum(res[-5:])/5

([0.7576991319656372,
  0.7459913492202759,
  0.7556630373001099,
  0.7411555051803589,
  0.7592262625694275],
 0.7519470572471618)

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.983266,1.791575,0.413591,0.870450,01:38
1,1.689537,1.573345,0.517944,0.928735,01:38
2,1.538139,1.434878,0.593026,0.943243,01:40
3,1.393348,1.309774,0.661746,0.951133,01:39
4,1.345759,1.233641,0.704250,0.962586,01:39
5,1.250168,1.157207,0.736065,0.966658,01:39
6,1.220388,1.106133,0.755918,0.969203,01:39
7,1.145580,1.103860,0.758208,0.970476,01:40
8,1.110340,1.084391,0.773225,0.968949,01:40
9,1.046738,1.067602,0.768898,0.972512,01:40


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.940531,1.779968,0.425808,0.880122,01:41
1,1.662416,1.593920,0.516925,0.918045,01:40
2,1.516636,1.398449,0.612115,0.939934,01:42
3,1.407049,1.304912,0.663782,0.955205,01:41
4,1.290138,1.187603,0.723848,0.964368,01:41
5,1.240288,1.161364,0.729957,0.965895,01:41
6,1.180779,1.164560,0.723848,0.963095,01:41
7,1.147595,1.087826,0.755409,0.972512,01:41
8,1.085288,1.038206,0.780860,0.972767,01:41
9,1.043810,1.034537,0.784678,0.974294,01:41


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.917777,1.803813,0.408246,0.879868,01:41
1,1.695563,1.633969,0.516416,0.919827,01:41
2,1.502566,1.369195,0.629422,0.948842,01:41
3,1.419279,1.268365,0.683380,0.956732,01:41
4,1.298121,1.192557,0.710868,0.967676,01:41
5,1.266458,1.158827,0.735811,0.964368,01:41
6,1.206960,1.140903,0.731229,0.967931,01:41
7,1.141000,1.091973,0.761517,0.974294,01:41
8,1.099991,1.094933,0.763808,0.971494,01:42
9,1.074507,1.065965,0.770425,0.974548,01:41


[0.749555, 0.757445, 0.759735, 0.761008, 0.751082, 0.848562, 0.850598, 0.85238, 0.851616, 0.856197] 0.8038177609443664


In [ ]:
res[5:10], sum(res[5:10])/5

([0.8485620021820068,
  0.8505980968475342,
  0.8523797392845154,
  0.8516162037849426,
  0.8561974763870239],
 0.8518707036972046)

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.121276,1.875138,0.403920,0.846271,01:42
1,1.874982,1.709075,0.468567,0.899975,01:41
2,1.763815,1.407820,0.609315,0.943497,01:41
3,1.622839,1.312779,0.660982,0.953423,01:41
4,1.516712,1.246700,0.695597,0.953932,01:40
5,1.493167,1.208635,0.692288,0.963095,01:41
6,1.453974,1.142897,0.743192,0.960804,01:41
7,1.400668,1.074391,0.773479,0.969203,01:41
8,1.366435,1.058389,0.776533,0.966658,01:42
9,1.311047,1.062316,0.772461,0.971749,01:42


[0.749555, 0.757445, 0.759735, 0.761008, 0.751082, 0.848562, 0.850598, 0.85238, 0.851616, 0.856197, 0.877068] 0.8104768720540133


In [ ]:
res = 
for epochs in [5]*5 + [20]*5 + [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.838385,1.767593,0.438789,0.889794,02:06
1,1.589668,1.530492,0.570374,0.931026,02:06
2,1.437728,1.305730,0.673199,0.958514,02:06
3,1.314570,1.167856,0.736065,0.960550,02:04
4,1.071313,1.033118,0.802240,0.979893,02:04


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


data path   /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.923147,1.754053,0.452023,0.894375,02:03
1,1.687327,1.558394,0.526597,0.932553,02:02
2,1.463223,1.375390,0.627641,0.945533,02:03
3,1.276294,1.230064,0.707814,0.960550,02:03
4,1.104879,1.067524,0.784169,0.975057,02:03


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.919139,1.818842,0.415627,0.882158,02:02
1,1.640623,1.503787,0.559430,0.930008,02:02


Buffered data was truncated after reaching the output size limit.

### epochs=5, 5 runs

no stride, bs=16, both act and bn

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.853887,1.693871,0.475185,0.900229,03:58
1,1.597855,1.421064,0.609824,0.941715,03:58
2,1.399992,1.231845,0.699160,0.966149,03:58
3,1.232522,1.124027,0.758463,0.966404,03:58
4,1.014952,1.001750,0.815220,0.981675,03:59


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.841401,1.678582,0.477475,0.906847,03:58
1,1.492334,1.419619,0.627641,0.952914,03:59
2,1.348735,1.279629,0.688216,0.963604,03:58
3,1.239337,1.132548,0.759735,0.972258,03:59
4,1.000990,0.989038,0.825146,0.981675,03:59


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.778878,1.642192,0.494273,0.912955,03:59
1,1.534141,1.428439,0.608552,0.942734,03:58
2,1.349526,1.314455,0.668872,0.956477,03:58
3,1.221146,1.120293,0.757699,0.973021,03:58
4,1.005900,0.979379,0.824128,0.980657,03:59


[0.81522, 0.814966, 0.826419, 0.825146, 0.824128] 0.8211758852005004


### epochs=20, 5 runs

no stride, bs=16, both act and bn

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.801173,1.698625,0.481547,0.901502,03:58
1,1.522946,1.387382,0.615933,0.950369,03:58
2,1.317431,1.288623,0.673963,0.955714,03:58
3,1.227739,1.129893,0.753881,0.970221,03:59
4,1.124146,1.070666,0.784424,0.976330,03:59
5,1.114767,1.045305,0.792823,0.975566,03:59
6,1.050665,1.001708,0.811402,0.976584,03:59
7,0.972466,0.985769,0.814966,0.983711,03:59
8,0.937151,0.992524,0.819292,0.980402,03:59
9,0.907048,0.947411,0.835073,0.980148,03:59


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.847284,1.775889,0.440825,0.892848,04:00
1,1.598621,1.442757,0.600153,0.949096,04:00
2,1.376522,1.265322,0.691779,0.956986,04:00
3,1.241434,1.172649,0.734793,0.962586,04:00
4,1.163452,1.083677,0.776533,0.975566,04:00
5,1.123032,1.051371,0.790023,0.975057,04:00
6,1.039444,1.001302,0.803767,0.979130,04:00
7,1.000067,0.971766,0.819292,0.982184,04:00
8,0.963111,1.012285,0.798167,0.975057,03:59
9,0.957597,0.925587,0.838381,0.982693,03:59


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.863371,1.890844,0.407228,0.878595,03:59
1,1.587108,1.468102,0.586663,0.942988,04:00
2,1.399580,1.308397,0.661491,0.957496,04:00
3,1.273798,1.182499,0.730466,0.966149,03:59
4,1.181075,1.145160,0.740392,0.969712,03:59
5,1.089572,1.120198,0.752609,0.973021,04:00
6,1.049326,1.013096,0.798422,0.979893,03:59
7,1.028194,1.039928,0.796131,0.974548,03:59
8,0.965961,0.990829,0.812420,0.976075,03:59
9,0.950209,0.971756,0.817765,0.980657,04:00


[0.890303, 0.88903, 0.890812, 0.890048, 0.886231] 0.8892848134040833


### epochs=80, 1 run

no stride, bs=16, no act, no bn

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.038643,1.808398,0.406974,0.869687,03:37
1,1.809108,1.668090,0.477984,0.896157,03:37
2,1.644071,1.380295,0.630695,0.947315,03:36
3,1.501918,1.250324,0.687961,0.955714,03:36
4,1.419764,1.168158,0.727157,0.965386,03:36
5,1.392190,1.110393,0.756936,0.974294,03:36
6,1.315425,1.051459,0.786969,0.975312,03:36
7,1.300064,1.040518,0.793841,0.974294,03:36
8,1.273304,0.996594,0.807330,0.979384,03:36
9,1.212061,0.999831,0.796386,0.976330,03:36


[0.901502] 0.9015016555786133



no stride, bs=16, both act and bn

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=128, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.009993,1.844606,0.379486,0.869432,03:59
1,1.781528,1.531310,0.555358,0.932807,03:58
2,1.613264,1.351579,0.640621,0.952405,03:59
3,1.511871,1.250022,0.708323,0.963095,03:59
4,1.425421,1.116004,0.758463,0.969458,03:59
5,1.373275,1.092476,0.767880,0.970730,03:58
6,1.300701,1.038407,0.790023,0.975312,03:58
7,1.282876,1.046970,0.786969,0.976330,03:58
8,1.268418,0.952731,0.826164,0.981675,03:58
9,1.188836,0.957474,0.818020,0.982438,03:58


[0.902011] 0.9020106792449951


## size=192

In [ ]:
dm = 2
res = dict()
for e in [5,80]: # [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.851319,1.850964,0.425554,0.870705,05:23
1,1.571606,1.486467,0.570629,0.937898,05:23
2,1.385945,1.273577,0.688979,0.962331,05:23
3,1.262263,1.225133,0.715195,0.961313,05:24
4,1.065164,1.007206,0.820565,0.979130,05:23


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/imagenette_experiments/train_utils.py:150: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


...

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.027898,1.831583,0.411810,0.867905,05:23
1,1.802953,1.590741,0.527106,0.921863,05:24
2,1.652540,1.390942,0.631713,0.950369,05:23
3,1.564943,1.250905,0.686689,0.966149,05:23
4,1.454332,1.172994,0.723085,0.965640,05:24
5,1.399670,1.112360,0.764571,0.969712,05:24
6,1.347242,1.047297,0.795368,0.978875,05:24
7,1.278714,1.071562,0.785696,0.975057,05:24
8,1.276993,0.990722,0.815984,0.978621,05:24
9,1.236502,1.004172,0.809112,0.980657,05:23


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


Buffered data was truncated after reaching the output size limit.

In [ ]:
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

depth multiplier=2 {5: 0.8205650448799133, 80: 0.9063374996185303}


In [ ]:
for i in range(80):
  print(i, learn.recorder.val_losses[i].item(), learn.recorder.metrics[i][0].item())

0 1.8315829038619995 0.41180962324142456
1 1.5907410383224487 0.5271061062812805
2 1.3909424543380737 0.6317129135131836
3 1.2509053945541382 0.6866887211799622
4 1.172993779182434 0.7230847477912903
5 1.1123597621917725 0.7645711302757263
6 1.0472970008850098 0.7953677773475647
7 1.0715622901916504 0.7856960892677307
8 0.9907216429710388 0.8159837126731873
9 1.0041720867156982 0.8091117143630981
10 0.9420099258422852 0.8424535393714905
11 0.9210571050643921 0.8432170748710632
12 0.9411424398422241 0.8340544700622559
13 0.9026457667350769 0.8521252274513245
14 0.8896369934082031 0.863069474697113
15 0.8706911206245422 0.8686688542366028
16 0.9016208648681641 0.8523797392845154
17 0.8971046805381775 0.8561974763870239
18 0.8691442012786865 0.8676508069038391
19 0.8719481229782104 0.863323986530304
20 0.858334481716156 0.8763043880462646
21 0.8563732504844666 0.8750318288803101
22 0.8597426414489746 0.8768134117126465
23 0.8590760827064514 0.86790531873703
24 0.850490927696228 0.87579536

In [ ]:
state = learn.model.state_dict()

In [ ]:
torch.save(state, 'drive/app/ResNet50_depth.pth')

In [ ]:
from google.colab import files
files.download('./checkpoint/ResNet50_depth.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dm = 2
res = dict()
for e in [5]*5 + [20]*5 + [80]:
  mixup=0 if e<=20 else 0.2
  learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
  learn.fit_fc(e, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
  acc = learn.recorder.metrics[-1][0].item()
  if e in res:
    res[e] += [acc]
  else:
    res[e] = [acc]
  print('{} epochs: {} ({} runs)'.format(e, sum(res[e])/len(res[e]), len(res[e])))
print('depth multiplier={}'.format(dm), {e: sum(res[e])/len(res[e]) for e in res})

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.881894,1.720682,0.470349,0.891575,02:02
1,1.574010,1.428916,0.607025,0.940952,02:02
2,1.405706,1.288442,0.680835,0.955205,02:03
3,1.289675,1.186716,0.724866,0.964877,02:02
4,1.075579,1.033660,0.789005,0.977857,02:02


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


...

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.025291,1.790612,0.421736,0.877322,02:03
1,1.853151,1.561010,0.534742,0.926190,02:04
2,1.677373,1.472112,0.584373,0.934589,02:04
3,1.572342,1.286127,0.668872,0.961059,02:04
4,1.486564,1.192217,0.718249,0.966658,02:04
5,1.401715,1.188251,0.725121,0.964368,02:05
6,1.341894,1.092382,0.766098,0.969203,02:04
7,1.311690,1.039083,0.783151,0.972003,02:04
8,1.298644,1.004073,0.805548,0.977093,02:03
9,1.245196,0.986797,0.804276,0.980148,02:03


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


80 epochs: 0.9030287861824036 (1 runs)
depth multiplier= {80: 0.9030287861824036}


experimenting with channel multiplier

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample wi

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.025110,1.872146,0.425554,0.871978,03:22
1,1.808593,1.560736,0.539577,0.931026,03:23
2,1.645650,1.374737,0.633240,0.948842,03:24
3,1.537533,1.259173,0.687452,0.959786,03:24
4,1.479700,1.155471,0.737338,0.969712,03:24
5,1.411763,1.107171,0.767625,0.968949,03:24
6,1.377335,1.080276,0.770425,0.969458,03:24
7,1.304939,1.025715,0.804021,0.973530,03:24
8,1.264274,0.998292,0.799949,0.977857,03:24
9,1.258094,0.983434,0.810130,0.981166,03:24


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/imagenette_experiments/train_utils.py:150: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


[0.907865] 0.9078645706176758


In [ ]:
res

[0.8961567878723145]



### epochs=5, 5 runs

no stride, bs=16, act=False, bn=False

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.856656,1.887479,0.400611,0.835836,07:11
1,1.616110,1.414912,0.620005,0.937898,07:11
2,1.392715,1.291010,0.676508,0.959786,07:11
3,1.269173,1.158866,0.738356,0.969203,07:11
4,1.050011,1.002696,0.822601,0.982438,07:11


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.850057,1.804008,0.418682,0.886740,07:10
1,1.552947,1.442660,0.597098,0.942479,07:11
2,1.384726,1.285309,0.679053,0.959023,07:10
3,1.249545,1.141736,0.743701,0.972258,07:11
4,1.064263,1.000224,0.822856,0.975566,07:11


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.851692,1.754275,0.448206,0.880122,07:10
1,1.579250,1.476122,0.580300,0.939679,07:11
2,1.406376,1.354866,0.640366,0.956477,07:11
3,1.256694,1.167863,0.746246,0.968694,07:11
4,1.020720,1.003255,0.817002,0.981420,07:11


[0.822601, 0.813184, 0.813948, 0.822856, 0.817002] 0.8179180502891541


no stride, bs=16, act=True, bn=False

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.816367,1.742989,0.460932,0.905574,07:18
1,1.570518,1.470981,0.579537,0.946806,07:19
2,1.335072,1.309832,0.661237,0.957496,07:18
3,1.251946,1.125895,0.758972,0.973021,07:18
4,1.014469,0.983681,0.824637,0.981675,07:18


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


...

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.821380,1.678589,0.481802,0.910919,07:18
1,1.630918,1.482963,0.576228,0.937643,07:18
2,1.379747,1.222471,0.704505,0.968440,07:18
3,1.262058,1.147718,0.755663,0.968949,07:18
4,1.063779,0.986779,0.833291,0.982947,07:19


data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.891106,1.707774,0.461441,0.902774,07:18
1,1.606799,1.477212,0.581064,0.935353,07:18
2,1.420447,1.298537,0.672436,0.959786,07:18
3,1.245861,1.184937,0.736829,0.969712,07:19
4,1.032215,0.994109,0.822601,0.978621,07:18


[0.824637, 0.829473, 0.824637, 0.833291, 0.822601] 0.8269279718399047


### epochs=20, 5 runs

no stride, bs=16, no act no bn

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=32, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

### epochs=80, 1 run

no stride, bs=16, no act no bn

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,2.012678,1.787375,0.416136,0.879104,07:11
1,1.827199,1.515741,0.567320,0.930262,07:11
2,1.661169,1.347738,0.643166,0.953678,07:11
3,1.538006,1.261420,0.697124,0.959023,07:11
4,1.459939,1.129074,0.749300,0.968185,07:12
5,1.386111,1.132712,0.755154,0.965640,07:11
6,1.344452,1.063014,0.784933,0.978112,07:12
7,1.331922,1.031150,0.806312,0.973021,07:12
8,1.276424,0.983552,0.819801,0.977348,07:12
9,1.231587,0.983760,0.818274,0.980657,07:11


Buffered data was truncated after reaching the output size limit.

Sorry, data was "truncated", but see the val_loss and accuracy below for proof. **92.08 is the all-time best that I've seen for Imagewoof2.**

In [ ]:
for i in range(80):
  print(i, learn.recorder.val_losses[i].item(), learn.recorder.metrics[i][0].item())

0 1.7873750925064087 0.41613641381263733
1 1.5157413482666016 0.567319929599762
2 1.347738265991211 0.643166184425354
3 1.2614200115203857 0.6971239447593689
4 1.129074215888977 0.7493000626564026
5 1.1327120065689087 0.7551539540290833
6 1.063014268875122 0.784932553768158
7 1.0311501026153564 0.8063120245933533
8 0.9835519194602966 0.8198014497756958
9 0.9837595820426941 0.8182743787765503
10 0.9428547024726868 0.833545446395874
11 0.95480877161026 0.8320183157920837
12 0.9107168316841125 0.8521252274513245
13 0.9013711214065552 0.8516162037849426
14 0.9020619988441467 0.8467803597450256
15 0.8675727844238281 0.8689233660697937
16 0.871545135974884 0.8620514273643494
17 0.897378146648407 0.8500890731811523
18 0.896333634853363 0.8653601408004761
19 0.8782687187194824 0.8620514273643494
20 0.8516641855239868 0.8791040778160095
21 0.8856545686721802 0.8513616919517517
22 0.8662623763084412 0.8722321391105652
23 0.8581082820892334 0.8732501864433289
24 0.8560348153114319 0.8735046982765

### epochs=200, 1 run

no stride at stem, bs=16, no act no bn

In [ ]:
res = []
for epochs in [200]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=192, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

data path   /root/.fastai/data/imagewoof2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

Learn path /root/.fastai/data/imagewoof2


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.991029,1.782373,0.423008,0.889030,07:12
1,1.820094,1.558511,0.531687,0.922372,07:11
2,1.640254,1.417206,0.610588,0.951642,07:11
3,1.524751,1.266035,0.692797,0.962077,07:11
4,1.451839,1.167214,0.736574,0.968440,07:12
5,1.363214,1.107141,0.767625,0.972767,07:12
6,1.331252,1.045224,0.792059,0.973530,07:12
7,1.325143,1.032004,0.803003,0.975821,07:12
8,1.271300,0.982155,0.821838,0.983965,07:12
9,1.266131,0.979309,0.827182,0.978875,07:12


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


In [ ]:
res

In [ ]:
for i in range(80):
  print(i, learn.recorder.val_losses[i].item(), learn.recorder.metrics[i][0].item())

## size=256

In [ ]:
for epochs in [5]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
for epochs in [20]*5:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))

In [ ]:
for epochs in [80]:
    mixup=0 if epochs<=20 else 0.2
    learn = get_learn(model=model, size=256, bs=16, mixup=mixup)
    learn.fit_fc(epochs, lr=4e-3, moms=(0.95,0.95), start_pct=0.72)
    res += [learn.recorder.metrics[-1][0].item()]
print([round(x, 6) for x in res], sum(res)/len(res))